In [2]:
# Cell 1: Imports & Setup

from pathlib import Path
import warnings
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from pathlib import Path

# Prophet import (supports both modern and legacy names)
try:
    from prophet import Prophet
except Exception:
    from fbprophet import Prophet  # fallback if older package is installed

# Optional visuals (off by default)
import plotly.express as px
import plotly.graph_objects as go

warnings.filterwarnings("ignore")

In [4]:

# Cell EP-1: Load Einstein productivity and compute human-demand series

# Canonical incoming path
INCOMING_PATH = r"C:\Users\pt3canro\Desktop\CAPACITY\Incoming_new.xlsx"

def build_daily_from_df(df_in: pd.DataFrame) -> pd.DataFrame:
    """Create a DAILY incoming series per vertical (tickets), filling missing days with zeros."""
    # Basic schema checks
    expected = {"Date", "vertical", "total_incoming"}
    missing = expected - set(df_in.columns)
    if missing:
        raise ValueError(f"Missing columns in Incoming_new.xlsx: {missing}")

    g = (df_in
         .assign(Date=pd.to_datetime(df_in["Date"], errors="coerce"))
         .dropna(subset=["Date"])
         .groupby(["vertical", pd.Grouper(key="Date", freq="D")])["total_incoming"]
         .sum()
         .rename("tickets")
         .reset_index())

    # Fill missing days per vertical (safe, no 'vertical' clash)
    g = (g.set_index("Date")
           .groupby("vertical", group_keys=False)
           .apply(lambda x: x.asfreq("D").fillna({"tickets": 0}))
           .reset_index())

    # Clean types
    g["tickets"] = g["tickets"].fillna(0).clip(lower=0).round().astype(int)
    # Force vertical to string labels (avoid float/NaN issues later)
    g["vertical"] = g["vertical"].astype(str).str.strip()
    return g.sort_values(["vertical", "Date"]).reset_index(drop=True)

# Rebuild 'df' if needed
if "df" not in globals():
    df = pd.read_excel(INCOMING_PATH)

# Rebuild 'daily' if needed
if "daily" not in globals():
    daily = build_daily_from_df(df)

# Quick sanity check
assert {"Date","vertical","tickets"} <= set(daily.columns), "'daily' lacks required columns"
display(daily.tail())



,Date,vertical,tickets
3155,2026-01-10,nan,0
3156,2026-01-10,nan,0
3157,2026-01-11,nan,0
3158,2026-01-11,nan,0
3159,2026-01-11,nan,0


In [7]:

# Cell CP-1: Load call performance and compute smoothed AHT per language

CALL_PATH = r"C:\Users\pt3canro\Desktop\CAPACITY\call_performance.xlsx"
cp = pd.read_excel(CALL_PATH)

cp["Date"] = pd.to_datetime(cp["Date"], errors="coerce")
cp = cp.dropna(subset=["Date"])

# Normalize column names
rename_map = {
    "lang": "language",
    "Language": "language",
    "AHT_sec": "aht_seconds",
    "aht": "aht_seconds",
}
cp.rename(columns={k:v for k,v in rename_map.items() if k in cp.columns}, inplace=True)

cp = cp[["Date","language","aht_seconds"]].dropna()
cp["aht_seconds"] = cp["aht_seconds"].clip(10, 3600)

# Smooth AHT with 14-day rolling median
cp = cp.sort_values(["language","Date"])
cp["aht_sm"] = cp.groupby("language")["aht_seconds"]\
                 .transform(lambda s: s.rolling(14, min_periods=3).median())




KeyError: 'Date'

In [ ]:

# Cell CP-2: Convert call AHT to ticket AHT
# Calls are usually shorter than tickets; use multiplier
TICKET_OVER_CALL = 1.35

call_aht = (cp.groupby("language")["aht_sm"]
              .median()
              .dropna()
              .to_dict())

ticket_aht_from_calls = {lang: float(aht) * TICKET_OVER_CALL 
                         for lang,aht in call_aht.items()}

ticket_aht_from_calls


In [ ]:

# Cell CAP-3: Integrate AHT from calls + Einstein for staffing

# 1. Use daily_with_bot instead of daily
df_staff = daily_with_bot.copy()

# 2. Allocate languages as before
# (reusing your Celda 8 logic)
df_staff["tickets_total"] = df_staff["tickets_human"]

alloc_rows = []
for _, row in df_staff.iterrows():
    alloc = allocate_by_language(int(row["tickets_total"]), CONFIG["language_shares"])
    for lang, t in alloc.items():
        alloc_rows.append({
            "Date": row["Date"],
            "vertical": row["vertical"],
            "language": lang,
            "tickets": int(t)
        })
df_lang_staff = pd.DataFrame(alloc_rows)

# 3. Replace AHT by call‑based values
df_lang_staff["aht_sec"] = df_lang_staff["language"].map(ticket_aht_from_calls).fillna(900)

# 4. Compute agents
WH = CONFIG["work_hours_effective"]
OCC = CONFIG["occupancy_target"]
SHR = CONFIG["shrinkage"]

df_lang_staff["agents"] = df_lang_staff.apply(
    lambda r: agents_needed(
        tickets=r["tickets"],
        aht_sec=r["aht_sec"],
        work_hours_effective=WH,
        occupancy=OCC,
        shrinkage=SHR
    ),
    axis=1
)

df_lang_staff.head()
``


In [ ]:

# Cell DASH-1: Final daily-by-language dashboard (incoming forecast → human demand → staffing → SLA gap)


# ----------------------------
# 0) Guards & parameters
# ----------------------------
REQUIRED_CONFIG_KEYS = ["language_shares", "work_hours_effective", "occupancy_target", "shrinkage", "export_dir"]
missing_cfg = [k for k in REQUIRED_CONFIG_KEYS if k not in CONFIG]
if missing_cfg:
    raise RuntimeError(f"CONFIG missing keys: {missing_cfg}. Please run your configuration cell first.")

LANG_SHARES = CONFIG["language_shares"].copy()
WORK_H = float(CONFIG["work_hours_effective"])
OCC   = float(CONFIG["occupancy_target"])
SHR   = float(CONFIG["shrinkage"])
OUTDIR = Path(CONFIG["export_dir"])
OUTDIR.mkdir(parents=True, exist_ok=True)

# Helper: deterministic Hamilton allocation
def allocate_by_language(total: int, shares: dict) -> dict:
    """Deterministic rounding (Hamilton method)."""
    total = int(total) if pd.notna(total) else 0
    if total <= 0:
        return {k: 0 for k in shares}
    ssum = float(sum(shares.values()))
    shares = {k: (v / ssum) for k, v in shares.items()} if not (0.99 <= ssum <= 1.01) else shares
    raw = {k: total * float(v) for k, v in shares.items()}
    base = {k: int(np.floor(v)) for k, v in raw.items()}
    remainder = int(total - sum(base.values()))
    if remainder > 0:
        fracs = sorted(((k, raw[k] - base[k]) for k in shares), key=lambda x: x[1], reverse=True)
        for k, _ in fracs[:remainder]:
            base[k] += 1
    return base

def agents_needed(tickets: int, aht_sec: int, work_h: float, occ: float, shrink: float) -> int:
    """Compute agents/day using ticket AHT, occupancy and shrinkage."""
    work_sec = work_h * 3600.0
    base = (tickets * aht_sec) / (work_sec * occ) if work_sec > 0 and occ > 0 else 0
    return int(np.ceil(base) * (1.0 + shrink))

# ----------------------------
# 1) Build incoming forecast by language (allocate from forecast_daily if not already available)
# ----------------------------
if "forecast_daily_lang" in globals():
    # Reuse precomputed language allocation
    incoming_lang = (forecast_daily_lang
                     .groupby(["Date","language"], as_index=False)["tickets"]
                     .sum()
                     .rename(columns={"tickets":"incoming_forecast"}))
else:
    if "forecast_daily" not in globals():
        raise RuntimeError("Missing 'forecast_daily'. Please run your forecasting cells first.")
    # Allocate forecast per language deterministically
    alloc_rows = []
    tmp = forecast_daily[["Date","vertical","tickets_total"]].copy()
    tmp["tickets_total"] = tmp["tickets_total"].fillna(0).clip(lower=0).round().astype(int)
    for _, row in tmp.iterrows():
        alloc = allocate_by_language(int(row["tickets_total"]), LANG_SHARES)
        for lang, t in alloc.items():
            alloc_rows.append({"Date": row["Date"], "language": lang, "incoming_forecast": int(t)})
    incoming_lang = (pd.DataFrame(alloc_rows)
                     .groupby(["Date","language"], as_index=False)["incoming_forecast"].sum())

# ----------------------------
# 2) Build Einstein baseline forecast by language
#    - If ein_daily exists (with or without language), project to forecast horizon (dates in incoming_lang)
#    - Otherwise, set to zero
# ----------------------------
# Collect forecast horizon dates
future_dates = pd.to_datetime(incoming_lang["Date"].unique())
min_future, max_future = future_dates.min(), future_dates.max()

def project_einstein_series(series: pd.Series, horizon_index: pd.DatetimeIndex) -> pd.Series:
    """
    Seasonal-naive (7-day) projection; fallback to rolling median if <7 obs; else zeros.
    'series' must have DateTimeIndex at daily freq for a language.
    """
    series = series.sort_index().asfreq("D").fillna(0)
    if len(series) >= 7:
        last_week = series.iloc[-7:].values
        reps = int(np.ceil(len(horizon_index)/7))
        fc = np.tile(last_week, reps)[:len(horizon_index)]
        return pd.Series(fc, index=horizon_index)
    elif len(series) >= 3:
        med = float(series.rolling(7, min_periods=3).median().iloc[-1])
        return pd.Series(np.full(len(horizon_index), med), index=horizon_index)
    else:
        return pd.Series(np.zeros(len(horizon_index)), index=horizon_index)

# Build einstein resolved forecast by language
if "ein_daily" in globals():
    # Try to see if ein_daily has language information
    ein_df = ein_daily.copy()
    if "language" in ein_df.columns:
        # aggregate per Date x language
        ein_hist = (ein_df.groupby(["Date","language"]).size()
                         .rename("einstein_resolved")
                         .reset_index())
    else:
        # no language -> aggregate total per day and allocate by language shares
        ein_hist = (ein_df.groupby("Date").size()
                         .rename("einstein_resolved")
                         .reset_index())
        alloc_rows_e = []
        for _, row in ein_hist.iterrows():
            alloc = allocate_by_language(int(row["einstein_resolved"]), LANG_SHARES)
            for lang, t in alloc.items():
                alloc_rows_e.append({"Date": row["Date"], "language": lang, "einstein_resolved": int(t)})
        ein_hist = pd.DataFrame(alloc_rows_e)

    ein_hist["Date"] = pd.to_datetime(ein_hist["Date"])
    # Project per language for the required horizon
    ein_fc_rows = []
    for lang in incoming_lang["language"].unique():
        hist_lang = (ein_hist[ein_hist["language"] == lang]
                        .set_index("Date")["einstein_resolved"])
        fc = project_einstein_series(hist_lang, pd.date_range(min_future, max_future, freq="D"))
        ein_fc_rows.append(pd.DataFrame({"Date": fc.index, "language": lang, "einstein_resolved_forecast": fc.values}))
    ein_fc = pd.concat(ein_fc_rows, ignore_index=True)
else:
    # No Einstein data provided -> assume zero for all horizon
    ein_fc = incoming_lang.copy()
    ein_fc["einstein_resolved_forecast"] = 0.0
    ein_fc = ein_fc[["Date","language","einstein_resolved_forecast"]]

# ----------------------------
# 3) Merge incoming forecast & Einstein forecast → human tickets
# ----------------------------
dash = (incoming_lang
        .merge(ein_fc, on=["Date","language"], how="left"))
dash["einstein_resolved_forecast"] = dash["einstein_resolved_forecast"].fillna(0).clip(lower=0)
dash["tickets_human"] = (dash["incoming_forecast"] - dash["einstein_resolved_forecast"]).clip(lower=0).astype(int)

# ----------------------------
# 4) Attach AHT per language
#    Priority: AHT_updated (from your CP-2 blend) → fallback CONFIG['aht_ticket_lang_sec'] → 900s
# ----------------------------
if "AHT_updated" in globals():
    AHT_MAP = {k:int(v) for k,v in AHT_updated.items()}
else:
    AHT_MAP = {k:int(v) for k,v in CONFIG.get("aht_ticket_lang_sec", {}).items()}

dash["aht_sec"] = dash["language"].map(AHT_MAP).fillna(900).astype(int)

# ----------------------------
# 5) Compute agents_needed per day & language
# ----------------------------
dash["agents_needed"] = dash.apply(
    lambda r: agents_needed(r["tickets_human"], r["aht_sec"], WORK_H, OCC, SHR),
    axis=1
)

# ----------------------------
# 6) SLA gap: if you provide df_planned_fte with columns [Date, language, fte_planned]
#    we compute closable tickets and the gap in agents & tickets.
# ----------------------------
if "df_planned_fte" in globals():
    plan = df_planned_fte.copy()
    plan["Date"] = pd.to_datetime(plan["Date"], errors="coerce")
    plan = plan.dropna(subset=["Date","language","fte_planned"])
    dash = dash.merge(plan[["Date","language","fte_planned"]], on=["Date","language"], how="left")
    dash["fte_planned"] = dash["fte_planned"].fillna(0.0)

    # Closable tickets with planned FTE: supply = FTE * work_sec * occ / AHT * (1 - shrink)
    work_sec = WORK_H * 3600.0
    dash["closable_tickets"] = ((dash["fte_planned"] * work_sec * OCC) / dash["aht_sec"]) * (1.0 - SHR)
    dash["closable_tickets"] = dash["closable_tickets"].fillna(0).round().astype(int)

    # Gaps
    dash["gap_tickets"] = (dash["tickets_human"] - dash["closable_tickets"]).clip(lower=0).astype(int)
    dash["gap_agents"]  = (dash["agents_needed"] - dash["fte_planned"]).clip(lower=0).round(0).astype(int)
else:
    dash["fte_planned"] = np.nan
    dash["closable_tickets"] = np.nan
    dash["gap_tickets"] = np.nan
    dash["gap_agents"] = np.nan

# ----------------------------
# 7) Nice ordering and export
# ----------------------------
dash = dash.sort_values(["Date","language"]).reset_index(drop=True)
cols = ["Date","language","incoming_forecast","einstein_resolved_forecast","tickets_human",
        "aht_sec","agents_needed","fte_planned","closable_tickets","gap_tickets","gap_agents"]
dash = dash[cols]

# Export CSV
out_csv = OUTDIR / "dashboard_daily_language.csv"
dash.to_csv(out_csv, index=False)
print("Saved dashboard:", out_csv)

# ----------------------------
# 8) Optional quick plots (stacked human tickets by language + total agents)
# ----------------------------
try:
    import plotly.express as px
    import plotly.graph_objects as go

    # Stacked bars: human tickets by language
    fig1 = px.area(dash, x="Date", y="tickets_human", color="language",
                   title="Human tickets per day by language (stacked)")
    fig1.show()

    # Total agents per day (sum across languages)
    agents_daily = dash.groupby("Date")["agents_needed"].sum().reset_index()
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=agents_daily["Date"], y=agents_daily["agents_needed"],
                              mode="lines+markers", name="Agents needed (total)"))
    fig2.update_layout(title="Total agents needed per day",
                       xaxis_title="Date", yaxis_title="Agents")
    fig2.show()
except Exception as e:
    print("Plotly not available or rendering blocked. Skipping plots. Reason:", e)

# Display head
dash.head(20)
